In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder,LabelEncoder
import keras
import tensorflow as tf
from keras import layers
from keras.models import load_model
from keras import activations,Model
from keras.utils import plot_model
from keras.models import load_model

In [2]:
df_egf = pd.read_csv(r'Database\Classification\EGFR.csv')
df_vegfr = pd.read_csv(r'Database\Classification\VEGFR2.csv')
df_fgfr = pd.read_csv(r'Database\Classification\FGFR2.csv')
df_drugs = pd.read_csv(r'Database\Drugbank.csv')

C:\Users\Mohammadreza\AppData\Local\Temp\ipykernel_12532\4138599882.py:4: DtypeWarning: Columns (286,287,288,289,290,291,292,293,294,295,1618,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1630,1631) have mixed types. Specify dtype option on import or set low_memory=False.
  df_drugs = pd.read_csv(r'C:\Users\Mohammadreza\Multi-target-Angiogenesis-Inhibitors\Database\Drugbank.csv')


In [3]:
minmax_scaler_egf=MinMaxScaler()
minmax_scaler_fgf=MinMaxScaler()
minmax_scaler_vegf=MinMaxScaler()
oneHot_encoder = OneHotEncoder()

In [4]:
def Standardization_egf(df):
    x_Data=df.iloc[:,1:].to_numpy()
    y_Data=df.iloc[:,0].to_numpy()
    minmax_scaler_egf.fit(x_Data)
    x_scale=minmax_scaler_egf.transform(x_Data)
    target = oneHot_encoder.fit_transform(y_Data.reshape(-1,1))
    return x_scale,y_Data

In [5]:
def Standardization_fgf(df):
    x_Data=df.iloc[:,1:].to_numpy()
    y_Data=df.iloc[:,0].to_numpy()
    minmax_scaler_fgf.fit(x_Data)
    x_scale=minmax_scaler_fgf.transform(x_Data)
    target = oneHot_encoder.fit_transform(y_Data.reshape(-1,1))
    return x_scale,y_Data

In [6]:
def Standardization_vegf(df):
    x_Data=df.iloc[:,1:].to_numpy()
    y_Data=df.iloc[:,0].to_numpy()
    minmax_scaler_vegf.fit(x_Data)
    x_scale=minmax_scaler_vegf.transform(x_Data)
    target = oneHot_encoder.fit_transform(y_Data.reshape(-1,1))
    return x_scale,target

In [7]:
def define_model(shape):
    encoding_dim=32
    input_vec=keras.Input(shape=(shape,))
    encoded=layers.Dense(1024,activation='relu')(input_vec)
    encoded=layers.Dropout(0.3)(encoded)
    encoded=layers.Dense(64,activation='relu')(encoded)
    encoded=layers.Dropout(0.2)(encoded)
    encoded=layers.Dense(2,activation='softmax')(encoded)
#     ------------------------------------------------------------
    model=keras.Model(input_vec,encoded)
    initial_learning_rate = 0.001
    opt = keras.optimizers.Adam(learning_rate=initial_learning_rate)
    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
#     model.save_weights(os.path.join(path,'initial_weight_Adam.h5'))
    
    return model

In [8]:
x_Data_egf,y_Data_egf=Standardization_egf(df_egf)
x_Data_fgf,y_Data_fgf=Standardization_fgf(df_fgfr)
x_Data_vegf,y_Data_vegf=Standardization_vegf(df_vegfr)

In [9]:
('egf:',x_Data_egf.shape,'fgf:',x_Data_fgf.shape,'vegf:',x_Data_vegf.shape)

('egf:', (8102, 2228), 'fgf:', (1345, 1926), 'vegf:', (8212, 2207))

In [10]:
df_drugs = df_drugs.replace('n.a.',np.nan)
df_drugs = df_drugs.dropna().reset_index(drop=True)

In [11]:
x_Drug_egf, x_Drug_vegfr, x_Drug_fgfr = minmax_scaler_egf.transform(df_drugs.iloc[:,:][df_egf.columns[1:]]), minmax_scaler_vegf.transform(df_drugs.iloc[:,:][df_vegfr.columns[1:]]), minmax_scaler_fgf.transform(df_drugs.iloc[:,:][df_fgfr.columns[1:]])

C:\Users\Mohammadreza\anaconda3\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\Mohammadreza\anaconda3\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\Mohammadreza\anaconda3\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


In [12]:
EGFR_classifier = define_model(x_Data_egf.shape[1])
EGFR_classifier.load_weights(r'Deep_Model\EGFR_model.h5')
EGFR_y_predicted_probability = EGFR_classifier.predict(x_Drug_egf)

58/58 [==============================] - 1s 11ms/step


In [13]:
VEGFR_classifier = define_model(x_Data_vegf.shape[1])
VEGFR_classifier.load_weights(r'Deep_Model\VEGFR2_model.h5')
VEGFR_y_predicted_probability = VEGFR_classifier.predict(x_Drug_vegfr)

58/58 [==============================] - 1s 16ms/step


In [14]:
FGFR_classifier = define_model(x_Data_fgf.shape[1])
FGFR_classifier.load_weights(r'Deep_Model\FGFR2_model.h5')
FGFR_y_predicted_probability = FGFR_classifier.predict(x_Drug_fgfr)

58/58 [==============================] - 1s 12ms/step


In [23]:
Final_probability = np.concatenate((df_drugs.MolID.values.reshape(-1,1),EGFR_y_predicted_probability,VEGFR_y_predicted_probability,FGFR_y_predicted_probability),axis=1)

In [24]:
# تعریف ستون‌ها با MultiIndex
columns = pd.MultiIndex.from_tuples([
    ('Drug', ''),
    ('EGFR', 'Inhibition Probability'),
    ('EGFR', 'Not Inhibition Probability'),
    ('VEGFR2', 'Inhibition Probability'),
    ('VEGFR2', 'Not Inhibition Probability'),
    ('FGFR2', 'Inhibition Probability'),
    ('FGFR2', 'Not Inhibition Probability'),
])

df = pd.DataFrame(Final_probability, columns=columns)

In [25]:
df

Drug                   EGFR                             \
              Inhibition Probability Not Inhibition Probability   
0     DB00006               0.551181                   0.448819   
1     DB00007               0.662543                   0.337457   
2     DB00014               0.713243                   0.286758   
3     DB00027               0.704255                   0.295745   
4     DB00035                0.37088                    0.62912   
...       ...                    ...                        ...   
1823  DB15982               0.633803                   0.366197   
1824  DB16022               0.639796                   0.360204   
1825  DB16625               0.293042                   0.706958   
1826  DB16627               0.315026                   0.684974   
1827  DB16628                0.92333                    0.07667   

                     VEGFR2                                             FGFR2  \
     Inhibition Probability Not Inhibition Probability Inhibition Probability   
0                  0.000039                   0.999961               0.802407   
1                  0.303699                   0.696301               0.954081   
2                   0.23973                    0.76027               0.966371   
3                  0.001759                   0.998241               0.914223   
4                  0.699943                   0.300057               0.998776   
...                     ...                        ...                    ...   
1823               0.571732                   0.428268               0.829672   
1824               0.939969                   0.060031               0.019564   
1825               0.352506                   0.647494               0.107199   
1826               0.679686                   0.320314               0.175099   
1827               0.999959                   0.000041               0.033296   

                                 
     Not Inhibition Probability  
0                      0.197593  
1                      0.045919  
2                      0.033629  
3                      0.085777  
4                      0.001223  
...                         ...  
1823                   0.170328  
1824                   0.980436  
1825                   0.892801  
1826                   0.824901  
1827                   0.966704  

[1828 rows x 7 columns]